In [0]:
%sql
CREATE OR REPLACE TEMP VIEW h1encounters
USING PARQUET
OPTIONS(
  path 'dbfs:/mnt/bronze/Hospital-1/Archive/2025/2/18/encounters'
);

CREATE OR REPLACE TEMP VIEW h2encounters
USING PARQUET
OPTIONS(
  path 'dbfs:/mnt/bronze/Hospital-2/Archive/2025/2/18/encounters'
);

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW h1encounters AS
SELECT *, 'Hospital-1' AS datasource
FROM PARQUET.`dbfs:/mnt/bronze/Hospital-1/Archive/2025/2/18/encounters`;

CREATE OR REPLACE TEMP VIEW h2encounters AS
SELECT *, 'Hospital-2' AS datasource
FROM PARQUET.`dbfs:/mnt/bronze/Hospital-2/Archive/2025/2/18/encounters`;

In [0]:
%sql
DESCRIBE h1encounters;

In [0]:
%sql
DESCRIBE h2encounters;

In [0]:
%sql
select * from h1encounters;

In [0]:
%sql
select * from h2encounters;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cmd_encounters
AS (
  SELECT * FROM h1encounters UNION ALL SELECT * FROM h2encounters
);

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW quality_checks AS
SELECT
CONCAT(EncounterID,'-',datasource) AS EncounterID,
EncounterID AS SRC_EncounterID,
PatientID,
EncounterDate,
EncounterType,
ProviderID,
DepartmentID,
ProcedureCode,
InsertedDate AS SRC_InsertedDate,
ModifiedDate AS SRC_ModifiedDate,
datasource,
CASE
WHEN EncounterID IS NULL OR PatientID IS NULL THEN TRUE
ELSE FALSE
END AS is_quarantined FROM cmd_encounters;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver.encounters(
  EncounterID STRING,
  SRC_EncounterID STRING,
  PatientID STRING,
  EncounterDate DATE,
  EncounterType STRING,
  ProviderID STRING,
  DepartmentID STRING,
  ProcedureCode STRING,
  SRC_InsertedDate DATE,
  SRC_ModifiedDate DATE,
  datasource STRING,
  is_quarantined BOOLEAN,
  audit_inserteddate TIMESTAMP,
  audit_modifieddate TIMESTAMP,
  is_current BOOLEAN
) USING DELTA;

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.encounters AS target
USING quality_checks AS source
ON target.EncounterID = source.EncounterID
AND target.is_current = TRUE
WHEN MATCHED AND(
  target.EncounterID <> source.EncounterID OR
  target.SRC_EncounterID <> source.SRC_EncounterID OR
  target.PatientID <> source.PatientID OR
  target.EncounterDate <> source.EncounterDate OR
  target.EncounterType <> source.EncounterType OR
  target.ProviderID <> source.ProviderID OR
  target.DepartmentID <> source.DepartmentID OR
  target.ProcedureCode <> source.ProcedureCode OR
  target.SRC_InsertedDate <> source.SRC_InsertedDate OR
  target.SRC_ModifiedDate <> source.SRC_ModifiedDate OR
  target.datasource <> source.datasource OR
  target.is_quarantined <> source.is_quarantined
)
THEN UPDATE SET is_current = FALSE,
        audit_modifieddate = current_timestamp();

In [0]:
%sql
MERGE INTO healthcarerevenuecyclemanagement_databricks.silver.encounters AS target
USING quality_checks AS source
ON target.EncounterID = source.EncounterID
AND target.is_current = TRUE
WHEN NOT MATCHED THEN INSERT(
  EncounterID,
  SRC_EncounterID,
  PatientID,
  EncounterDate,
  EncounterType,
  ProviderID,
  DepartmentID,
  ProcedureCode,
  SRC_InsertedDate,
  SRC_ModifiedDate,
  datasource,
  is_quarantined,
  audit_inserteddate,
  audit_modifieddate,
  is_current
) VALUES (
  source.EncounterID,
  source.SRC_EncounterID,
  source.PatientID,
  source.EncounterDate,
  source.EncounterType,
  source.ProviderID,
  source.DepartmentID,
  source.ProcedureCode,
  source.SRC_InsertedDate,
  source.SRC_ModifiedDate,
  source.datasource,
  source.is_quarantined,
  current_timestamp(),
  current_timestamp(),
  TRUE
);

In [0]:
%sql
DESCRIBE healthcarerevenuecyclemanagement_databricks.silver.encounters;
SELECT * FROM healthcarerevenuecyclemanagement_databricks.silver.encounters;

In [0]:
%sql
SELECT * FROM healthcarerevenuecyclemanagement_databricks.silver.encounters where datasource = 'Hospital-1';